In [62]:
import folium #Umożliwia tworzenie interaktywnych map w języku Python
from IPython.display import display # Umożliwia wyświetlanie interaktywnych wyników w środowisku IPython
from geopy.geocoders import Nominatim #służy do geokodowania, czyli przekształcania adresów lub miejsc na współrzędne geograficzne.
import requests
import polyline # używana do reprezentacji tras na mapie.
import pandas as pd
from folium.plugins import MarkerCluster, FeatureGroupSubGroup

In [2]:
#WNETRZNOSCI DO RECZNEGO PUSZCZANIA GOTOWA FUNKCJA NA SAMYM KONCU PODAJEMY W NIEJ DF !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [87]:
data = pd.read_csv('./sonar_training.csv', index_col=0)
data['ulica'] = data['ulica'].astype(str) + ', Wrocław'
streets = list(data.ulica)

In [88]:
df = pd.DataFrame(data=streets[17:24],columns=['ulica'])

In [89]:
df

,ulica
0,"Zamkowa, Wrocław"
1,"Zamkowa, Wrocław"
2,"Rymarska, Wrocław"
3,"Trzebnicka, Wrocław"
4,"Woskowa, Wrocław"
5,"Woskowa, Wrocław"
6,"Konińska, Wrocław"


In [90]:
punktacja = [50,43,95,86,67,77,69]

In [91]:
df['punktacja']=punktacja 

In [8]:
df=pd.concat([df,pd.DataFrame(data=[['plac Grunwaldzki 2, Wrocław',999]],columns=['ulica','punktacja'])],ignore_index=True) #dodanie im jako startu

In [9]:
df=df.sort_values(by='punktacja',ascending=False)
df

,ulica,punktacja
7,"plac Grunwaldzki 2, Wrocław",999
2,"Rymarska, Wrocław",95
3,"Trzebnicka, Wrocław",86
5,"Woskowa, Wrocław",77
6,"Konińska, Wrocław",69
4,"Woskowa, Wrocław",67
0,"Zamkowa, Wrocław",50
1,"Zamkowa, Wrocław",43


In [10]:
kolorki = ['blue','green','green','green','orange','orange','orange','orange']
ikonki = ['search','home','home','home','map-marker','map-marker','map-marker','map-marker']

In [11]:
df['kolor']=kolorki
df['ikona']=ikonki

In [84]:
#tutaj jakies ogolne ustawienia zeby mapka wro sie pojawila, ewentualnie zmieniac zoom_start
mapbox_api_key = 'pk.eyJ1Ijoid3ljZW55IiwiYSI6ImNscnRraDBsajA3NXQyam5hdjloZTBoZ3EifQ.0gU_ozTPH70_eXfyfCAqkA'

geolocator = Nominatim(user_agent="my_geocoder") #definiujemy usługę geokodowania
addresses = df #zeby nie zmieniac zmiennych, nadpisuje

loc = geolocator.geocode('Wrocław')
loc = [loc.latitude,loc.longitude]

map_wroclaw = folium.Map(loc, zoom_start=12, control_scale=True)
# Dodawanie warstwy mapbox do mapy
folium.TileLayer(
    tiles=f'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}',
    attr='Mapbox',
    name='Mapbox Streets',
    overlay=True
).add_to(map_wroclaw)


In [85]:
coordinates=[]
adres = []
	
marker_cluster = MarkerCluster().add_to(map_wroclaw)

for i in range(0,8): #stawianie 3 glownych punktow i IM, i zbieranie koordynatow wszystkich
    adres.append(df.iloc[i]['ulica'])
    kolor = df.iloc[i]['kolor']
    ikona = df.iloc[i]['ikona']
    location = geolocator.geocode(adres[i])
    if i <= 3: #zeby na razie dodalo tylko im i 3 glowne punkty
        folium.Marker(location=[location.latitude, location.longitude], popup=adres[i],icon=folium.Icon(color=kolor, icon=ikona)).add_to(marker_cluster)
    coordinates.append([location.longitude,location.latitude])


#szukanie optymalnej drogi miedzy im a trzema najlepszymi
profile = 'driving'
optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]};{coordinates[3][0]},{coordinates[3][1]}?access_token={mapbox_api_key}'
response = requests.get(optimized_trips_url)
data_opt = response.json()
dystans = round(data_opt['trips'][0]['distance']/1000,2)
czas = round(data_opt['trips'][0]['duration']/60)
#ponizej sprawdzamy dla pozostalych 4 opcjonlanych miejsc czy dolozenie ich 
#zwiekszy czas przejazdu o poonad 20% - jesli tak olewamy, jesli nie dokladamy.
czas_dodatkowy=[]
for i in range(4,8):
    profile = 'driving'
    optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]};{coordinates[3][0]},{coordinates[3][1]};{coordinates[i][0]},{coordinates[i][1]}?access_token={mapbox_api_key}'
    response = requests.get(optimized_trips_url)
    data_opt = response.json()
    if data_opt['trips'][0]['duration']/60 <= czas*1.2:
        czas_dodatkowy.append([data_opt['trips'][0]['duration']/60 - czas,i])
#sprawdzanie ile znalazlo i tworzenie z nimi nowej drogi
#wpisywanie wspolrzednych do stringa i wrzucanie znacznikow dodatkowych miejsc na mape
pozostale = '' 
for i in range (0,len(czas_dodatkowy)):
    pozostale = pozostale + ';' +str(coordinates[czas_dodatkowy[i][1]][0])+','+str(coordinates[czas_dodatkowy[i][1]][1])
    folium.Marker(location=[coordinates[czas_dodatkowy[i][1]][1], coordinates[czas_dodatkowy[i][1]][0]],popup=adres[czas_dodatkowy[i][1]],icon=folium.Icon(color='orange', icon='map-marker')).add_to(marker_cluster)

#oficjalna trasa z punktami ktore sie oplaca
optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]};{coordinates[3][0]},{coordinates[3][1]}{pozostale}?access_token={mapbox_api_key}'

#reszta rzeczy zeby mapa sie pojawila i stringi z informacja ile sie jedzie bez / z punktami dodatkowymi.
response = requests.get(optimized_trips_url)
data_opt = response.json()
coor = polyline.decode(data_opt['trips'][0]['geometry'])
folium.PolyLine(locations=coor, color='blue').add_to(map_wroclaw)
print(f"Odległość trasy bez dodatkowych ofert: ok. {dystans} km")
print(f"Czas podróży bez dodatkowych ofert: ok. {czas} min")
print(f"Odległość trasy: ok. {round(data_opt['trips'][0]['distance']/1000,2)} km")
print(f"Czas podróży: ok. {round(data_opt['trips'][0]['duration']/60)} min")


display(map_wroclaw)
map_wroclaw.save('map.html')

Odległość trasy bez dodatkowych ofert: ok. 29.24 km
Czas podróży bez dodatkowych ofert: ok. 85 min
Odległość trasy: ok. 29.81 km
Czas podróży: ok. 90 min


In [93]:
def rysowanie_mapki(df):
    '''Aktualnie df (z 7 ofertami) ma miec postac tabelki na pewno z kolumnami 'ulica' -> 'ulica, Wroclaw', oraz 'punktacja' od 0 do 100, IM jest dopisywany automatycznie. Df w srodku sortuje sie wzgledem punktacji i ustawia kolorki ikonki itd. '''
    df=pd.concat([df,pd.DataFrame(data=[['plac Grunwaldzki 2, Wrocław',999]],columns=['ulica','punktacja'])],ignore_index=True) #dodanie im jako startu
    kolorki = ['blue','green','green','green','orange','orange','orange','orange']
    ikonki = ['search','home','home','home','map-marker','map-marker','map-marker','map-marker']
    df=df.sort_values(by='punktacja',ascending=False)
    df['kolor']=kolorki
    df['ikona']=ikonki

    mapbox_api_key = 'pk.eyJ1Ijoid3ljZW55IiwiYSI6ImNscnRraDBsajA3NXQyam5hdjloZTBoZ3EifQ.0gU_ozTPH70_eXfyfCAqkA'

    geolocator = Nominatim(user_agent="my_geocoder") #definiujemy usługę geokodowania
    addresses = df
    
    loc = geolocator.geocode('Wrocław')
    loc = [loc.latitude,loc.longitude]
    
    map_wroclaw = folium.Map(loc, zoom_start=12, control_scale=True)
    # Dodawanie warstwy mapbox do mapy
    folium.TileLayer(
        tiles=f'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}',
        attr='Mapbox',
        name='Mapbox Streets',
        overlay=True
    ).add_to(map_wroclaw)
    

    coordinates=[]
    adres = []
    	
    marker_cluster = MarkerCluster().add_to(map_wroclaw)
    
    for i in range(0,8): #stawianie 3 glownych punktow i IM, i zbieranie koordynatow wszystkich
        adres.append(df.iloc[i]['ulica'])
        kolor = df.iloc[i]['kolor']
        ikona = df.iloc[i]['ikona']
        location = geolocator.geocode(adres[i])
        if i <= 3: #zeby na razie dodalo tylko im i 3 glowne punkty
            folium.Marker(location=[location.latitude, location.longitude], popup=adres[i],icon=folium.Icon(color=kolor, icon=ikona)).add_to(marker_cluster)
        coordinates.append([location.longitude,location.latitude])
    
    
    #szukanie optymalnej drogi miedzy im a trzema najlepszymi
    profile = 'driving'
    optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]};{coordinates[3][0]},{coordinates[3][1]}?access_token={mapbox_api_key}'
    response = requests.get(optimized_trips_url)
    data_opt = response.json()
    dystans = round(data_opt['trips'][0]['distance']/1000,2)
    czas = round(data_opt['trips'][0]['duration']/60)
    #ponizej sprawdzamy dla pozostalych 4 opcjonlanych miejsc czy dolozenie ich 
    #zwiekszy czas przejazdu o poonad 20% - jesli tak olewamy, jesli nie dokladamy.
    czas_dodatkowy=[]
    for i in range(4,8):
        profile = 'driving'
        optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]};{coordinates[3][0]},{coordinates[3][1]};{coordinates[i][0]},{coordinates[i][1]}?access_token={mapbox_api_key}'
        response = requests.get(optimized_trips_url)
        data_opt = response.json()
        if data_opt['trips'][0]['duration']/60 <= czas*1.2:
            czas_dodatkowy.append([data_opt['trips'][0]['duration']/60 - czas,i])
    #sprawdzanie ile znalazlo i tworzenie z nimi nowej drogi
    #wpisywanie wspolrzednych do stringa i wrzucanie znacznikow dodatkowych miejsc na mape
    pozostale = '' 
    for i in range (0,len(czas_dodatkowy)):
        pozostale = pozostale + ';' +str(coordinates[czas_dodatkowy[i][1]][0])+','+str(coordinates[czas_dodatkowy[i][1]][1])
        folium.Marker(location=[coordinates[czas_dodatkowy[i][1]][1], coordinates[czas_dodatkowy[i][1]][0]],popup=adres[czas_dodatkowy[i][1]],icon=folium.Icon(color='orange', icon='map-marker')).add_to(marker_cluster)
    
    #oficjalna trasa z punktami ktore sie oplaca
    optimized_trips_url = f'https://api.mapbox.com/optimized-trips/v1/mapbox/{profile}/{coordinates[0][0]},{coordinates[0][1]};{coordinates[1][0]},{coordinates[1][1]};{coordinates[2][0]},{coordinates[2][1]};{coordinates[3][0]},{coordinates[3][1]}{pozostale}?access_token={mapbox_api_key}'
    
    #reszta rzeczy zeby mapa sie pojawila i stringi z informacja ile sie jedzie bez / z punktami dodatkowymi.
    response = requests.get(optimized_trips_url)
    data_opt = response.json()
    coor = polyline.decode(data_opt['trips'][0]['geometry'])
    folium.PolyLine(locations=coor, color='blue').add_to(map_wroclaw)
    print(f"Odległość trasy bez dodatkowych ofert: ok. {dystans} km")
    print(f"Czas podróży bez dodatkowych ofert: ok. {czas} min")
    print(f"Odległość trasy: ok. {round(data_opt['trips'][0]['distance']/1000,2)} km")
    print(f"Czas podróży: ok. {round(data_opt['trips'][0]['duration']/60)} min")
    
    
    map_wroclaw.save('map.html')
    return display(map_wroclaw)

In [94]:
rysowanie_mapki(df)

Odległość trasy bez dodatkowych ofert: ok. 29.24 km
Czas podróży bez dodatkowych ofert: ok. 85 min
Odległość trasy: ok. 29.81 km
Czas podróży: ok. 90 min
